In [ ]:
import os
import numpy as np
from pycanoe import CanoeDataset
from pycanoe.utils.utils import get_inverse_tf
from pycanoe.utils.vis_utils import bilinear_interp, plot_points_on_img, convert_to_bev

In [ ]:
canoe_root = '/path/to/canoe'
dataset = CanoeDataset(canoe_root)

for idx, seq in enumerate(dataset.sequences): print(f"id: {idx}, name: {seq.ID}")

In [ ]:
seq = dataset.sequences[0]

# Note: doesn't necessarily "synchronize" frames of different sensors, just grabs closest in time
seq.synchronize_frames()

In [ ]:
# Visualize frames at same index
idx = 0

# Camera
cam = seq.camleft_frames[idx]
cam.load_data()
cam.visualize()
cam.unload_data()

# Radar
rad = seq.radar_frames[idx]
rad.load_data()
rad.visualize()
rad.unload_data()

# Lidar
lid = seq.lidar_frames[idx]
lid.load_data()
lid.visualize()
lid.unload_data()

# Sonar
sonar = seq.sonar_frames[idx]
sonar.load_data()
sonar.visualize()
sonar.unload_data()

## Lidar-Cam Overlay

In [ ]:
max_depth=50

In [ ]:
idx=360 # For sample sequence (canoe-2025-08-21-19-16)

cam = seq.camleft_frames[idx]
lid = seq.lidar_frames[idx]
rad = seq.radar_frames[idx]

In [ ]:
_ = cam.load_data()
_ = lid.load_data()

# Lidar: transform to cam, keep only forward-facing
lid.transform(seq.calib.T_camleft_lidar)
lid.passthrough([-100, 100, -100, 100, 0, max_depth]) #  [xmin, xmax, ymin, ymax, zmin, zmax]

# Project lidar points onto image, colored by depth
lid_on_cam_uv, lid_on_cam_depth, lid_on_cam_mask = lid.project_onto_image(seq.calib.P)
overlay1 = plot_points_on_img(cam.img, lid_on_cam_uv, lid_on_cam_depth)

# Img colors at lidar points
lid_on_cam_uv_color = bilinear_interp(cam.img, lid_on_cam_uv[:,0], lid_on_cam_uv[:,1])/255.0
overlay2 = plot_points_on_img(np.ones(cam.img.shape), lid_on_cam_uv, lid_on_cam_uv_color, s=100, alpha=1)

cam.unload_data()
lid.unload_data()

## Project Lidar onto Radar

In [ ]:
cart_pixel_width = 640
cart_resolution = 0.25
max_height = 1

In [ ]:
_ = rad.load_data()
_ = lid.load_data()

# Radar cartesian form
rad_cart = rad.polar_to_cart(cart_resolution, cart_pixel_width, in_place=False)
rad_cart = np.repeat(rad_cart[:,:,np.newaxis], 3, axis=2)

# Transform lidar to radar frame
lid.transform(get_inverse_tf(seq.calib.T_lidar_radar))
# Filter to a maximum height deviation from radar
lid.passthrough([-999, 999, -999, 999, -max_height, max_height]) #  [xmin, xmax, ymin, ymax, zmin, zmax]
# Convert to BEV
lid_uv, mask = convert_to_bev(lid.points[:,0:2], cart_resolution, cart_pixel_width)
lid_depth = np.sqrt(lid.points[:,0]**2 + lid.points[:,1]**2)[mask]

# Radar alone
rad.visualize(cart_resolution=cart_resolution, cart_pixel_width=cart_pixel_width)
# Lidar (colorized by depth) on radar
plot_points_on_img(rad_cart, lid_uv, lid_depth, alpha=0.75, s=4)

rad.unload_data()
lid.unload_data()